In [1]:
using PCT

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


## $x^{\dagger} A x$

In [2]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j)))
blaserize(f) |> latex |> println

A \to \mathcal{P}\left(x \to x^{\dagger}\cdot A\cdot x\right)


In [3]:
df = redux(vdiff(f), settings=symmetry_settings) |> blaserize |> latex |> println

A \to x \to 2.0 \cdot A\cdot x


## Trace of Product

In [39]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::N, j::N) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [40]:
eval_all(f)

𝒫((A, B) -> 
    ∑((i), (∑((n), (A(n, i)⋅B(i, n)))+∑((n), (A(i, n)⋅B(n, i))))))

In [42]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)  |> blaserize

(A, B) -> 
    2.0⋅B⁺, 2.0⋅A⁺

In [43]:
codegen(df)

:((A, B)->begin
          #= /Users/likangbo/kaer_morhen/projects/QLD/PCT/src/codegen.jl:58 =#
          #= /Users/likangbo/kaer_morhen/projects/QLD/PCT/src/codegen.jl:60 =#
          [Expr[:(2.0 .* transpose(conj(B))), :(2.0 .* transpose(conj(A)))]...]
      end)

## Conjugate Gradient

In [100]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::N) -> x(i) + α * (r(i) + β * p(i))))
end; blaserize(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        𝒫((α, β) -> 
            E((i) -> 
                (x(i)+(r(i)+p(i)⋅β)⋅α)))
    end

In [101]:
df = vdiff(f) |> blaserize 

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        (α, β) -> 
            𝒫(E)((i) -> 
                (x(i)+(r(i)+p(i)⋅β)⋅α), 1)ᵀ⋅(β⋅p+r), α⋅𝒫(E)((i) -> 
                (x(i)+(r(i)+p(i)⋅β)⋅α), 1)ᵀ⋅p
    end

In [12]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize |> latex |> println

\left(A, r, p, b, x\right) \to \left(α, β\right) \to \left(\left(\left(r+β \cdot p\right)^{T}\cdot b+\left(x+α \cdot \left(r+β \cdot p\right)\right)^{T}\cdot A\cdot \left(r+β \cdot p\right)\right), α\cdot \left(b^{T}\cdot p+\left(x+α \cdot \left(r+β \cdot p\right)\right)^{T}\cdot A\cdot p\right)\right)


## Lagrangian Mechanics

In [53]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [54]:
f, _ = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [55]:
redux(vdiff(f); settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        ((_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1)+D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1)))

## Hartree Fock Gradient and Hessian Vector Product

In [14]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [15]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [16]:
df = redux(vdiff(f); settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            (4.0⋅∑((i, p, q, r), (C(q, i)'⋅J(_d, r, q, p)⋅C(p, i)⋅C(r, _d_1))))

In [17]:
df |> get_body |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a, _a_1) -> 
        (∑((_d, i, p, q), (J(_d, _a, q, p)'⋅C(p, i)'⋅4.0⋅_k(_d, _a_1)⋅C(q, i)))+∑((_d, _d_1, q, r), (J(_d, r, q, _a)'⋅C(r, _d_1)'⋅4.0⋅_k(_d, _d_1)⋅C(q, _a_1)))+∑((_d, _d_1, p, r), (4.0⋅J(_d, r, _a, p)⋅_k(_d, _d_1)⋅C(p, _a_1)⋅C(r, _d_1))))

## Foster Boys

In [25]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::N) -> sum((p, q), R(p, q) * C(p, i)' * C(q, i))
        sum(i, sum((p, q), R2(p, q) * C(p, i)' * C(q, i))) - sum(i, r(i)' * r(i))
    end); blaserize(f)

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                (C⁺⋅R⋅C)(i, i)
            (tr(C⋅C⁺⋅R2)(-1.0⋅r⁺⋅r))
        end)

In [27]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

(R, R2) -> 
    (C) -> 
        (-4.0⋅((_d, _d_1) -> 
            ((R⋅C)(_d, _d_1)⋅(C⁺⋅R⋅C)(_d_1, _d_1)))+2.0⋅R2⋅C)

## Localized Wannier Functions

In [8]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (N, N, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> R
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (N, N, BZ) -> C
    end
end;

In [9]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::N, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), w(b) * ρ(n, b)' * ρ(n, b))
    end); f

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)')
            ∑((n, b), w(b)⋅ρ(n, b)⋅ρ(n, b)')
        end)

In [11]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))

df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

1 sum_dist
∑((n, b), (∑((k, p, q), ((_d, _d_1, _d_2) -> 
    δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, ∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)')⋅w(b)⋅U(p, n, k)⋅S(p, q, k, (b+k))')))+(_d, _d_1, _d_2) -> 
    ∑((_z_1), δ(k, _d_2, δ(n, _d_1, δ(p, _d, _z_1⋅S(p, q, k, (b+k))⋅δ(_z_1, U(q, n, (b+k)), ∑((k, p, q), U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (b+k))')⋅w(b))))))))+∑((_z_3, k, p, q), ((_d, _d_1, _d_2) -> 
    δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, _z_3⋅w(b)⋅U(p, n, k)⋅S(p, q, k, (b+k))'⋅δ(_z_3, ∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), 1))))+(_d, _d_1, _d_2) -> 
    ∑((_z_2), δ(k, _d_2, δ(n, _d_1, δ(p, _d, S(p, q, k, (b+k))⋅_z_2'⋅δ(U(q, n, (b+k))', _z_2, w(b)⋅_z_3'⋅δ(∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), _z_3, 1))))))))))
Add
(∑((n, b, k, p, q), ((_d, _d_1, _d_2) -> 
    δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, ∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)')⋅w(b)⋅U(p, n, k)⋅S(p, q, k, (b+k))')))+(_d, _d_1, _d_2) -> 
    ∑((_z_1), δ(k, 

7 contract_delta
(_d, _d_1, _d_2) -> 
    (∑((n, b, p, q), δ(n, _d_1, δ(q, _d, ∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)')⋅w(b)⋅U(p, n, (-1.0⋅b+_d_2))⋅S(p, q, (-1.0⋅b+_d_2), _d_2)')))+∑((n, b, _z_3, k, p, q), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, _z_3⋅w(b)⋅U(p, n, k)⋅S(p, q, k, (b+k))'⋅δ(_z_3, ∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), 1)))))+∑((n, b, k, p, q, _z_1), δ(k, _d_2, δ(n, _d_1, δ(p, _d, _z_1⋅S(p, q, k, (b+k))⋅δ(_z_1, U(q, n, (b+k)), ∑((k, p, q), U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (b+k))')⋅w(b))))))+∑((n, b, _z_3, k, p, q, _z_2), δ(k, _d_2, δ(n, _d_1, δ(p, _d, S(p, q, k, (b+k))⋅_z_2'⋅δ(U(q, n, (b+k))', _z_2, w(b)⋅_z_3'⋅δ(∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), _z_3, 1)))))))
Map
(_d, _d_1, _d_2) -> 
    (∑((b, p, q), δ(q, _d, ∑((k, p, q), U(q, _d_1, (b+k))⋅S(p, q, k, (b+k))⋅U(p, _d_1, k)')⋅w(b)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(p, q, (-1.0⋅b+_d_2), _d_2)'))+∑((n, b, _z_3, k, p, q), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, _z_3⋅w(b)⋅U(p,

13 swallow
(_d, _d_1, _d_2) -> 
    (∑((_z_3, b, p), _z_3⋅w(b)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅δ(_z_3, ∑((k, p, q), U(q, _d_1, (b+k))⋅S(p, q, k, (b+k))⋅U(p, _d_1, k)'), 1))+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')+∑((n, b, k, p, q, _z_1), δ(k, _d_2, δ(n, _d_1, δ(p, _d, _z_1⋅S(p, q, k, (b+k))⋅δ(_z_1, U(q, n, (b+k)), ∑((k, p, q), U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (b+k))')⋅w(b))))))+∑((n, b, _z_3, k, p, q, _z_2), δ(k, _d_2, δ(n, _d_1, δ(p, _d, S(p, q, k, (b+k))⋅_z_2'⋅δ(U(q, n, (b+k))', _z_2, w(b)⋅_z_3'⋅δ(∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), _z_3, 1)))))))
Map
(_d, _d_1, _d_2) -> 
    (∑((_z_3, b, p), δ(∑((k, p, q), U(q, _d_1, (b+k))⋅S(p, q, k, (b+k))⋅U(p, _d_1, k)'), _z_3, _z_3⋅w(b)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S

21 sum_in
(_d, _d_1, _d_2) -> 
    (∑((b, q), ∑((k, p, _q), U(p, _d_1, k)⋅U(_q, _d_1, (b+k))'⋅S(p, _q, k, (b+k))')⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅S(_d, q, _d_2, (_d_2+b)))+∑((n, b, _z_3, k, p, q, _z_2), δ(k, _d_2, δ(n, _d_1, δ(p, _d, S(p, q, k, (b+k))⋅_z_2'⋅δ(U(q, n, (b+k))', _z_2, w(b)⋅_z_3'⋅δ(∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), _z_3, 1))))))+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')⋅2.0)
Map
(_d, _d_1, _d_2) -> 
    (∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b))⋅U(_q, _d_1, (b+k))'⋅S(p, _q, k, (b+k))')+∑((n, b, _z_3, k, p, q, _z_2), δ(k, _d_2, δ(n, _d_1, δ(p, _d, S(p, q, k, (b+k))⋅_z_2'⋅δ(U(q, n, (b+k))', _z_2, w(b)⋅_z_3'⋅δ(∑((k, p, q), U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'), _z_3, 1))))))+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')⋅2.0)

1 shift
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b))⋅U(_q, _d_1, (b+k))'⋅S(p, _q, k, (b+k))')+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))⋅2.0
Mul
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, (k-1.0⋅b))⋅S(_d, q, _d_2, (_d_2+b))⋅U(_q, _d_1, k)'⋅S(p, _q, (k-1.0⋅b), k)')+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))⋅2.0
<->

2 sum_sym
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, (k-1.0⋅b))⋅S(_d, q, _d_2, (_d_2+b))⋅U(_q, _d_1, k)'⋅S(p, _q, (k-1.0⋅b), k)')+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))⋅2.0
Mul
(∑((b, q, k, p, _q), w(-1.0⋅b)⋅U(q, _d_1, (-1.0⋅b+_d_2))⋅U(p, _d_1, (b+k))⋅S(_d, q, _d_2, (-1.0⋅b+_d_2))⋅U(_q, _d_1, k)'⋅S(p, _q, (b+k), k)')+∑((b, p

16 symmetry
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅U(_q, _d_1, (b+k))'⋅S(q, _d, (_d_2+b), _d_2)'⋅S(p, _q, k, (b+k))')+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))⋅2.0
Mul
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_q, p, (b+k), k)⋅U(_q, _d_1, (b+k))'⋅S(q, _d, (_d_2+b), _d_2)')+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))⋅2.0
<->

17 shift
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_q, p, (b+k), k)⋅U(_q, _d_1, (b+k))'⋅S(q, _d, (_d_2+b), _d_2)')+∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'))⋅2.0
Mul
(∑((b, q, k, p, _q), w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, (k-1.0⋅b))⋅S(_q, p, k, (k-1.0⋅b))⋅U(_q, _d_1, k)'⋅S(q, _d, (_d_2+b), _d_2)')+∑((b, p, k, _i, q), w(b)⋅U

39 shift
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, k)'⋅S(q, _i, (b+k), k)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, (k-1.0⋅b))'⋅S(q, _i, k, (k-1.0⋅b))')
<->

40 sum_sym
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, (k-1.0⋅b))'⋅S(q, _i, k, (k-1.0⋅b))')
Sum
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅U(_i, _d_1, (b+k))'⋅S(q, _i, k, (b+k))')
<->

41 shift
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅U(_i, _d_1, (b+k))'⋅S(q, _i, k, (b+k))')
Sum
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅U(_i, _d_1, k)'⋅S(q, _i, (k-1.0⋅b), k)')
<->

42 symmetry
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, 

69 symmetry
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, k)'⋅S(q, _i, (b+k), k)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, k)'⋅S(q, _i, (b+k), k)')
<->

70 symmetry
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, k)'⋅S(q, _i, (b+k), k)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)')
<->

71 symmetry
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅U(_i, _d_1, (k-1.0⋅b))'⋅S(q, _i, k, (k-1.0⋅b))'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅U(_i, _d_1, (k-1.0⋅b))'⋅S(q, _i, k, (k-1.0⋅b))')
<->

72 symmetry
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2

98 symmetry
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, (k-1.0⋅b), k)⋅U(_i, _d_1, (k-1.0⋅b))'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_d, p, _d_2, (-1.0⋅b+_d_2))⋅S(_i, q, (k-1.0⋅b), k)⋅U(_i, _d_1, (k-1.0⋅b))')
<->

99 sum_sym
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_i, q, k, (k-1.0⋅b))⋅U(_i, _d_1, k)'⋅S(p, _d, (_d_2+b), _d_2)')
<->

100 symmetry
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')
Sum
∑((b, p, k, _i, q), 4.0⋅w(-1.0⋅b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_i, q, k, (b+k))⋅U(_i, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)')
<->

101 symmetry
∑((b, p, k, _i, q), 4.0⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            ∑((b, p, k, _i, q), w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_i, q, (b+k), k)⋅U(_i, _d_1, (b+k))')⋅4.0

In [23]:
_, ctx = @pct begin
    @space Tau begin
        type=(R, RV) -> RM
    end
    
    @space Nu begin
        type=(RV, ) -> R
    end
        
    @space S begin
        type=(RV, )->RV
    end
end;

In [66]:
f, _ = @pct _ ctx (τ::Tau, ν::Nu, p::S) -> :vdiff(pullback((x::RV) -> τ(ν(x), p(x)))); f

(τ, ν, p) -> 
    vdiff̂(𝒫((x) -> 
        τ(ν(x), p(x))))

In [15]:
df = process_directive(f)

(τ, ν, p) -> 
    (x) -> 
        (𝒫(ν)(x, 𝒫((_z) -> 
            τ(_z, p(x)))(ν(x), 1))+𝒫(p)(x, 𝒫((_z_1) -> 
            τ(ν(x), _z_1))(p(x), 1)))

In [2]:
_, ctx = @pct begin
    @space G begin
        type=(RV,)->R
    end
    end;

In [3]:
f, _ = @pct _ ctx (wstar::RV) -> (g::G) -> pullback((w::RV)  -> (g(w) - g(wstar))^2)
f

(wstar) -> 
    (g) -> 
        𝒫((w) -> 
            ((-1.0⋅g(wstar)+g(w)))^(2))

In [4]:
df = vdiff(f)

(wstar) -> 
    (g) -> 
        (w) -> 
            𝒫(g)(w, 2.0⋅(-1.0⋅g(wstar)+g(w)))

In [18]:
hf = df |> get_body |> get_body |> decompose |> pp

(w, _k) -> 
    (∑((_z), 𝒫((_z_2) -> 
        𝒫(g)(_z_2, _z))(w, δ(2.0⋅(-1.0⋅g(wstar)+g(w)), _z, _k)))+𝒫(g)(w, 2.0⋅(_z) -> 
        𝒫(g)(w, _z)⁺(_k)))

In [29]:
hf |> simplify |> first

(w, _k) -> 
    (𝒫(g)(w, 2.0⋅(_z) -> 
        𝒫(g)(w, _z)ᵀ(_k))+𝒫((_z_2) -> 
        𝒫(g)(_z_2, 2.0⋅(-1.0⋅g(wstar)+g(w))))(w, _k))